# Poligonization

## 0 Initialization

### 0.0 Import Libraries

In [1]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import copy
import trimesh as tm
import pandas as pd
import boolean_marching_cubes as bmc

### 0.1 Generate Symmetry Stencils

In [2]:
# example symmetry strings
##########################
# sym_str = [["OO"], ["XP"], ["XN"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
sym_str = [["OO"], ["XX"], ["YY"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX", "YY"], ["ZZ"]]
# sym_str = [["OO"], ["XX", "YY", "ZZ"]]
# sym_str = [["OO"]]

stencils = bmc.create_symmetry_stencils(sym_str)

### 0.2 Generate Lattices for all possible cubes

In [3]:
# generate bianary representation of all the possible cubes
l_bis = bmc.bi_cube_lattices()

## 1 Profiling

### 1.1 Construct profile latices

In [4]:
# find all unique corner arrangements based on stencils
corner_profiles = bmc.extract_corner_profiles(stencils, l_bis)

# stack corner_profiles vertically
cp_stacked = np.vstack(corner_profiles)

# find the uniqe arangements of corners
uniq_corner_arang = np.unique(cp_stacked, axis=0)

# construct lattices for all unique corner profiles
(corner_loc_lattices, corner_neigh_lattices) = bmc.profiles_to_lattices(uniq_corner_arang, stencils)

#show the ammount of unique profiles
print(len(uniq_corner_arang))

24


### 1.3. Save unique arrangement profiles

In [5]:
# save all design templates into lattice CSVs
templates_path = os.path.relpath('../data/bmc/bmc_templates_2')
bmc.save_design_templates(corner_loc_lattices, corner_neigh_lattices, templates_path)

## 2 Construct the tile-set

### 2.1. Load sub-tile meshes

In [6]:
# load subtile meshes for either set
subtile_meshes = []
for c in range(len(corner_loc_lattices)):
    corner_mesh_path = os.path.relpath('../data/bmc/selfdesigned_subtiles_1/t_' + f'{c:02}' + '.obj')
    corner_mesh = tm.load(corner_mesh_path)
    subtile_meshes.append(corner_mesh)

unable to load materials from: t_12.mtl


### 2.2. Combine sub-tile meshes to create tile meshes

might be the problem?
    This is different to what they did last year. Why did it change?
    How does this work?

In [7]:
def construct_tile_meshes(subtile_meshes, corner_profiles, uniq_corner_arang, corner_loc_lattices):
    tile_corner_inds = []
    for prof in corner_profiles:
        # find each corner in the list of unique corner arrangements
        corner_ind = np.array(
            [np.where((p == uniq_corner_arang).all(1)) for p in prof]).flatten()
        tile_corner_inds.append(corner_ind)

    tile_corner_inds = np.array(tile_corner_inds)
    # print(tile_corner_inds)

    ###############################

    # find unique locs

    unique_locs = [np.array(np.where(loc == 1)).flatten()
                   for loc in corner_loc_lattices]
    unique_locs = np.array(unique_locs)

    ###############################

    corner_loc = np.array(np.where(np.ones((2, 2, 2)) == 1)).T
    corner_pos = corner_loc - .5
    # print(corner_pos)

    tiles_meshes = []
    # loading meshes
    for tile in tile_corner_inds:
        last_v_count = 0
        vertice_list = []
        face_list = []
        for c_ind, pos, loc in zip(tile, corner_pos, corner_loc):

            # extract current mesh
            corner_mesh = subtile_meshes[c_ind]
            # extract the unique profile of this corner
            u_loc = unique_locs[c_ind]
            if type(corner_mesh) == tm.base.Trimesh:
                # if current profile is different than unique profile, we need a transformation, -1 will flip over that dimension and 1 will keep it the same
                trans = (u_loc - loc) * 2 + 1
                mirror_normal = u_loc - loc
                if np.any(mirror_normal < 0) : 
                    Mx = tm.transformations.reflection_matrix([0,0,0], mirror_normal)
                    vertices_transformed = tm.transformations.transform_points(corner_mesh.vertices, Mx)
                else:
                    vertices_transformed = corner_mesh.vertices
                # append the vertices
                vertice_list.append(vertices_transformed + pos)
                face_list.append(corner_mesh.faces + last_v_count)
                last_v_count += len(corner_mesh.vertices)

        vs = []
        fs = []
        if len(vertice_list):
            vs = np.vstack(vertice_list)
            fs = np.vstack(face_list)

            tile_mesh = tm.Trimesh(vs * 0.5, fs)
            tiles_meshes.append(tile_mesh)
        else:
            empty_tile = tm.creation.icosphere(subdivisions=1, radius=0.1)
            tiles_meshes.append(empty_tile)

    return tiles_meshes


tiles_meshes = construct_tile_meshes(subtile_meshes, corner_profiles, uniq_corner_arang, corner_loc_lattices)

### 2.3. Save the tile-set

In [8]:
tiles_path = os.path.relpath('../data/bmc/bmc_tiles_3')
bmc.save_tile_meshes(tiles_meshes, l_bis, tiles_path)

### 2.3. Visualize the tile set

In [17]:
tiles_set = []
for c in range(256):
    tiles_meshes_set_path = os.path.relpath('../data/bmc/bmc_tiles_3/t_' + f'{c:03}' + '.obj')
    corner_mesh_set = tm.load(tiles_meshes_set_path)
    tiles_set.append(corner_mesh_set)

# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

p = pv.Plotter(notebook=True)

base_lattice = l_bis[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5 
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit *0.5

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    i = int(value)
    mesh = tiles_set[i]
    lattice = l_bis[i]

    # Add the data values to the cell data
    grid.cell_arrays["cube"] = lattice.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="cube")
#     # adding the voxels
#     p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=0.2, show_scalar_bar=False, color="white")

    # adding the meshes
    p.add_mesh(tri_to_pv(mesh), color='#abd8ff', name="sphere")

    return

p.add_slider_widget(create_mesh, [0, 256], title='Tiles', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(2.3763431631744796, 2.3763431631744796, 2.1888431631744796),
 (-0.0625, -0.0625, -0.25),
 (0.0, 0.0, 1.0)]

## 3. Boolean Marching Cube

### 3.1 Load envelope lattice

In [10]:
# loading the lattice from csv
solar_envelope_path = ('../data/meshes/useable_lattice.csv')
envelope_lattice = tg.lattice_from_csv(solar_envelope_path)

#Pad to make sure all voxels are tiled
envelope_lattice_expanded = np.pad(envelope_lattice, ((1,1),(1,1),(1,1)),'constant',constant_values= 0)
envelope_lattice_expanded = tg.to_lattice(envelope_lattice_expanded, envelope_lattice.minbound-envelope_lattice.unit, envelope_lattice.unit)
envelope_lattice = envelope_lattice_expanded

### 3.1 Remove voxels that are not on the boundary

In [11]:
#Remove voxels that are not on the boundry

# create the stencil
s = tg.create_stencil("von_neumann", 1, 1)
s.set_index([0,0,0], 0)

# add the sum function to the stencil
s.function = tg.sfunc.sum 

# apply the stencil on the lattice
neighbor_sum = envelope_lattice.apply_stencil(s)

# remove the voxel if less than 6 neighbours
new_envelope_lattice = envelope_lattice * (neighbor_sum <= 5)

### 3.2. Extract the cube lattice from the envelope lattice

In [12]:
cube_lattice = new_envelope_lattice.boolean_marching_cubes()
np.set_printoptions(threshold=100000)
# print(cube_lattice)

### 3.3. Select which voxels to tile

whill most likely be usefull with multiple materials

In [13]:
#Select which column of voxels will be tiled.
#Set all other voxels to 0, to isolate the tiling
#cube_lattice[:,5:,:] = 0
#cube_lattice[:,:4,:] = 0

### 3.4. Tile the cube lattice with a selected tileset

In [20]:
#Choosing which tileset to tile with and un-commenting it

#The first set
tileset_path = os.path.relpath('../data/bmc/bmc_tiles_3')
#The second set
# tileset_path = os.path.relpath('../data/bmc/bmc_tiles_Set2')

bmc_mesh = bmc.marching_cube_mesh(cube_lattice, tileset_path)

### 3.5. Visualize the final mesh

In [21]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(bmc_mesh), color='#abd8ff', name="sphere")

# fast visualization of the lattice
p = envelope_lattice.fast_vis(p)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(257.3028852194484, 164.80288521944837, 239.80288521944837),
 (35.0, -57.5, 17.5),
 (0.0, 0.0, 1.0)]